Main figure of the Alborex Data paper in ESSD journal:         
positions of all the deployments (to keep it simple and clear).

In [38]:
import os
import glob
import json
import logging
import cmocean
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib import colors
import datetime, time, calendar
import matplotlib.text as text
from matplotlib.path import Path
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import alborexdata
from importlib import reload

In [39]:
reload(alborexdata)

<module 'alborexdata' from '/home/ctroupin/Publis/201703_AlborexData/python/alborexdata.py'>

# Configuration
Most of the configuration (file paths, domain, resolution, ...) is done in the json file [`alborexconfig.json`](./alborexconfig.json).

In [40]:
with open('alborexconfig.json') as json_data_file:
    config = json.load(json_data_file)

## Logger

In [41]:
logger = alborexdata.configure_logging("./logs/alborexFigure1.log")

### Figures

In [109]:
figdir = config["figdir"]
figname = os.path.join(figdir, 'fig01')
figname0 = os.path.join(figdir, 'fig00')

## Time interval for the drifters

In [43]:
drifterstart = datetime.datetime(2014, 5, 25, 0, 0, 0)
drifterend = datetime.datetime(2014, 6, 15, 0, 0, 0)

## Domain

In [44]:
coordinates = config["domain"]["coordinates1"]
coordinates2 = config["domain"]["coordinates2"]
dlon, dlat = config["domain"]["coordinates1"][0], config["domain"]["coordinates1"][1]

# Load data
## Coast

In [45]:
loncoast, latcoast = alborexdata.read_lonlat_coast(config["datafiles"]["coast"])

## Ship

In [46]:
rv = alborexdata.Ship()
rv.get_from_netcdf(config["datafiles"]["rv"])
rv.apply_qc()

## CTD

In [47]:
ctd = alborexdata.CTD()
ctd.get_from_netcdf(config["datafiles"]["ctd"])
lonCTD1, latCTD1 = alborexdata.load_lonloat_ctdleg(config["datafiles"]["ctdlegs"][0])

## Gliders

In [48]:
glider1 = alborexdata.Glider()
glider1.get_from_netcdf(config["datafiles"]["gliders"][0])
glider2 = alborexdata.Glider()
glider2.get_from_netcdf(config["datafiles"]["gliders"][1])

## Drifters

In [49]:
drifterdir = sorted(config["datafiles"]["drifters"])

## Profilers

In [91]:
profiler1 = alborexdata.Profiler()
profiler2 = alborexdata.Profiler()
profiler3 = alborexdata.Profiler()
profiler1.get_from_netcdf(config["datafiles"]["profilers"][0])
profiler2.get_from_netcdf(config["datafiles"]["profilers"][1])
profiler3.get_from_netcdf(config["datafiles"]["profilers"][2])

In [101]:
# Modity the year (error in L1 data for ArvorC, waiting for a correction)
for i in range(0, len(profiler3.dates)):
    profiler3.dates[i] = profiler3.dates[i].replace(year=2014)

In [102]:
profiler1.select_dates(initialdate=datetime.datetime(2014, 5, 30, 8, 0),
                       finaldate=drifterend)
profiler2.select_dates(initialdate=drifterstart,
                       finaldate=drifterend)
profiler3.select_dates(initialdate=drifterstart,
                       finaldate=drifterend)

In [84]:
with netCDF4.Dataset(config["datafiles"]["profilers"][2], "r") as nc:
    ttime = nc.get_variables_by_attributes(standard_name='time')[0][:]
    timeunits = nc.get_variables_by_attributes(standard_name='time')[0].units
ddates = netCDF4.num2date(ttime, timeunits)
timeunits

'seconds since 1970-01-01 00:00:00'

## SST
The files could not be added to the GitHub repos, but can be downloaded from [OceanColor](http://oceancolor.gsfc.nasa.gov/) website.      The URLs can be changed in the [configuration file](alborexconfig.json) if necessary.

In [55]:
sstfilelist = config["datafiles"]["sst"]
sstfilelistremote = config["datafiles"]["sstremote"]
for sstfile, sstremote in zip(sstfilelist, sstfilelistremote):
    if not(os.path.exists(sstfile)):
        logger.info("Downloading data file {}".format(os.path.basename(sstfile)))
        urllib.request.urlretrieve(sstremote, sstfile)
    else:
        logger.info("SST file {} already downloaded".format(os.path.basename(sstfile)))
lonsst, latsst, sst, sstqual, year, day, platform = alborexdata.load_sst_l2(sstfilelist[0])
# Make the SST when flag > 1
sst = np.ma.masked_where(sstqual > 1, sst)

2019-01-09 21:30:22,565 - alborex_logger - INFO - SST file A2014145125000.L2_LAC_SST.nc already downloaded
2019-01-09 21:30:22,565 - alborex_logger - INFO - SST file A2014145125000.L2_LAC_SST.nc already downloaded
2019-01-09 21:30:22,568 - alborex_logger - INFO - SST file A2014150020500.L2_LAC_SST.nc already downloaded
2019-01-09 21:30:22,568 - alborex_logger - INFO - SST file A2014150020500.L2_LAC_SST.nc already downloaded


## Front position

In [52]:
frontcoords = config["datafiles"]["front"]
f = alborexdata.Front()
f.get_from_file(frontcoords)
f.smooth()

# Create the figure

In [112]:
m = Basemap(projection="merc",
            llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
            urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
            lat_ts=0.5 * (coordinates[2] + coordinates[3]), resolution="h")